In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d

def analyze_fabric_tpi(image_path, ppi=None, roi_size=1024):
    """
    Analyzes a fabric image to calculate Warp and Weft TPI. This definitive version
    correctly finds the peak closest to the center for maximum accuracy.
    """
    # --- 1. Load and Preprocess Image ---
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Error: Could not load image from {image_path}")
        return

    h, w = img.shape
    if h < roi_size or w < roi_size:
        print(f"Error: Image is smaller than the specified ROI size of {roi_size}x{roi_size}.")
        return

    start_h = (h - roi_size) // 2
    start_w = (w - roi_size) // 2
    roi = img[start_h : start_h + roi_size, start_w : start_w + roi_size]

    # --- 2. Perform 2D Fast Fourier Transform (FFT) ---
    f_transform = np.fft.fft2(roi)
    f_transform_shifted = np.fft.fftshift(f_transform)
    magnitude_spectrum = np.log1p(np.abs(f_transform_shifted))

    # --- 3. Find Peaks in the Spectrum (DEFINITIVE LOGIC) ---
    center_x, center_y = roi_size // 2, roi_size // 2

    vertical_line = magnitude_spectrum[:, center_x]
    horizontal_line = magnitude_spectrum[center_y, :]

    smoothed_vertical_line = gaussian_filter1d(vertical_line, sigma=2)
    smoothed_horizontal_line = gaussian_filter1d(horizontal_line, sigma=2)

    weft_peaks, _ = find_peaks(smoothed_vertical_line, prominence=0.1, distance=5)
    warp_peaks, _ = find_peaks(smoothed_horizontal_line, prominence=0.1, distance=5)

    center_ignore_radius = 20

    # For Weft
    valid_weft_indices = np.where(np.abs(weft_peaks - center_y) > center_ignore_radius)[0]
    weft_freq = 0
    if valid_weft_indices.size > 0:
        # Find the valid peak closest to the center
        valid_peak_locations = weft_peaks[valid_weft_indices]
        distances_from_center = np.abs(valid_peak_locations - center_y)
        closest_peak_index = np.argmin(distances_from_center)
        weft_peak_y = valid_peak_locations[closest_peak_index]
        weft_freq = np.min(distances_from_center)

    # For Warp
    valid_warp_indices = np.where(np.abs(warp_peaks - center_x) > center_ignore_radius)[0]
    warp_freq = 0
    if valid_warp_indices.size > 0:
        # Find the valid peak closest to the center
        valid_peak_locations = warp_peaks[valid_warp_indices]
        distances_from_center = np.abs(valid_peak_locations - center_x)
        closest_peak_index = np.argmin(distances_from_center)
        warp_peak_x = valid_peak_locations[closest_peak_index]
        warp_freq = np.min(distances_from_center)


    # --- 4. Calculate and Print Results ---
    print(f"Detected Weft Frequency (distance from center): {weft_freq:.2f} pixels")
    print(f"Detected Warp Frequency (distance from center): {warp_freq:.2f} pixels")

    if ppi:
        roi_size_inches = roi_size / ppi
        weft_tpi = weft_freq / roi_size_inches
        warp_tpi = warp_freq / roi_size_inches
        print("\n--- Final Results (TPI) ---")
        print(f"Weft (Horizontal Threads): {weft_tpi:.2f} TPI")
        print(f"Warp (Vertical Threads): {warp_tpi:.2f} TPI")
    else:
        print("\n--- Final Results (Threads per ROI) ---")
        print("PPI not provided. Displaying raw thread count across the ROI.")
        print(f"Weft (Horizontal Threads): {weft_freq:.2f} threads")
        print(f"Warp (Vertical Threads): {warp_freq:.2f} threads")

    # --- 5. Visualization ---
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(roi, cmap='gray')
    plt.title(f'Cropped ROI ({roi_size}x{roi_size})')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(magnitude_spectrum, cmap='gray')
    if warp_freq > 0:
        plt.plot(warp_peak_x, center_y, 'ro', markersize=8, label=f'Warp Peak (Freq: {int(warp_freq)})')
    if weft_freq > 0:
        plt.plot(center_x, weft_peak_y, 'bo', markersize=8, label=f'Weft Peak (Freq: {int(weft_freq)})')
    plt.title('FFT Magnitude Spectrum')
    plt.legend()
    plt.axis('off')

    plt.tight_layout()
    plt.show()

KeyboardInterrupt: 